In [16]:
%%javascript
var width = window.innerWidth || document.documentElement.clientWidth || document.body.clientWidth;
var height = window.innerHeight || document.documentElement.clientHeight || document.body.clientHeight;

IPython.notebook.kernel.execute("windowSize = (" + width + "," + height + ")");
// suitable for small screens
nbpresent.mode.tree.set(
    ["app", "theme-manager", "themes", "my-theme"], 
    {
    palette: {
        "blue": { id: "blue", rgb: [0, 153, 204] },
        "black": { id: "black", rgb: [0, 0, 0] },
        "white": { id: "white", rgb: [255, 255, 255] },
        "red": { id: "red", rgb: [240, 32, 32] },
        "gray": { id: "gray", rgb: [128, 128, 128] },
    },
    backgrounds: {
        "my-background": {
            "background-color": "white"
        }
    },
    "text-base": {
        "font-family": "Georgia",
        "font-size": 2.5
    },
    rules: {
        h1: {
            "font-size": 5.5,
            color: "blue",
            "text-align": "center"
        },
        h2: {
            "font-size": 3,
            color: "blue",
            "text-align": "center"
        },
        h3: {
            "font-size": 3,
            color: "black",
        },
        "ul li": {
            "font-size": 2.5,
            color: "black"
        },
        "ul li ul li": {
            "font-size": 2.0,
            color: "black"
        },
        "code": {
            "font-size": 1.6,
        },
        "pre": {
            "font-size": 1.6,
        }
    }
});


<IPython.core.display.Javascript object>

In [17]:
from IPython.core.display import display, HTML
display(HTML("<style> *{margin:0; padding:0;} html, body, .container{margin:0;!important padding:0;!important} .container { width:100% !important;}</style>"))

# Suffix Arrays and BWTs
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22Backwards.jpg" width="500px">

* 2<sup>nd</sup> problem set due on Thursday
* Midterm next Thursday, this is the last lecture that will be covered on the midterm
<p style="text-align: right; clear: right;">1</p>

# A tweak to argsort()

* Recall argsort() from last time:

``` python
def argsort(input):
    return sorted(range(len(input)), cmp=lambda i,j: 1 if input[i] >= input[j] else -1)

B = ["TAGACAT", "AGACAT", "GACAT", "ACAT", "CAT", "AT", "T"]
print argsort(B)
```

* If we know that our input is suffixes from a single string
    - the $i^{th}$ suffix starts at index i
    - thus we don't need to extract the suffixes, just use offsets

<p style="text-align: right; clear: right;">2</p>

# Constructing a Suffix Array

Note the small change to the argsort() function (*cmp* uses [i:], [j:] rather than [i], [j]) to compute a suffix array directly from the given text.

In [18]:
def suffixArray(text):
    return sorted(range(len(text)), cmp=lambda i,j: 1 if text[i:] >= text[j:] else -1)

t = "amanaplanacanalpanama"
sa = suffixArray(t)
print sa
for i in sa:
    print "%2d: %s" % (i, t[i:])

[20, 9, 13, 18, 0, 7, 11, 16, 2, 4, 10, 6, 14, 19, 1, 8, 12, 17, 3, 15, 5]
20: a
 9: acanalpanama
13: alpanama
18: ama
 0: amanaplanacanalpanama
 7: anacanalpanama
11: analpanama
16: anama
 2: anaplanacanalpanama
 4: aplanacanalpanama
10: canalpanama
 6: lanacanalpanama
14: lpanama
19: ma
 1: manaplanacanalpanama
 8: nacanalpanama
12: nalpanama
17: nama
 3: naplanacanalpanama
15: panama
 5: planacanalpanama


<p style="text-align: right; clear: right;">3</p>

# Searching a Suffix Array

* Searching a sorted list requires $O(log(m))$ comparisons using ***binary search***
* Each comparision is over *n* symbols of the pattern
* Thus, searching is $O(n log(m))$

In [19]:
def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:] < pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

first = findFirst("an", t, sa)
print t
print first, sa[first], t[sa[first]:]

amanaplanacanalpanama
5 7 anacanalpanama


<p style="text-align: right; clear: right;">4</p>

# Finding all Occurences

A variant to binary search which finds the last occurence of a pattern rather than the first. Only difference, uses "<=" instead of "<", but needs to trim string comparison to test for equality.

In [20]:
def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)/2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

print t
last = findLast("an", t, sa)
print first, last
for suffix in sa[first:last]:     # recall "first" was found on the previous slide
    print "%3d: %s" % (suffix, t[suffix:])
print last - first, "times"

amanaplanacanalpanama
5 9
  7: anacanalpanama
 11: analpanama
 16: anama
  2: anaplanacanalpanama
4 times


<p style="text-align: right; clear: right;">5</p>

# Longest repeated substring?

* Given a suffix array, we can compute a *helper* function, call the **Longest Common Prefix**, LCP

In [21]:
def computeLCP(text, suffixarray):
    m = len(text)
    lcp = [0 for i in xrange(m)]
    for i in xrange(1,m):
        u = suffixarray[i-1]
        v = suffixarray[i]
        n = 0
        while text[u] == text[v]:
            n += 1
            u += 1
            v += 1
            if (u >= m) or (v >=m):
                break
        lcp[i] = n
    return lcp

lcp = computeLCP(t, sa)

print "SA,LCP,Suffix"
for i, j in enumerate(sa):
    print "%2d: %2d %s" % (j, lcp[i], t[j:])

SA,LCP,Suffix
20:  0 a
 9:  1 acanalpanama
13:  1 alpanama
18:  1 ama
 0:  3 amanaplanacanalpanama
 7:  1 anacanalpanama
11:  3 analpanama
16:  3 anama
 2:  3 anaplanacanalpanama
 4:  1 aplanacanalpanama
10:  0 canalpanama
 6:  0 lanacanalpanama
14:  1 lpanama
19:  0 ma
 1:  2 manaplanacanalpanama
 8:  0 nacanalpanama
12:  2 nalpanama
17:  2 nama
 3:  2 naplanacanalpanama
15:  0 panama
 5:  1 planacanalpanama


* It should be evident that the longest repeated substring is at the suffix array index with the largest LCP
* It is shared with one or more suffixes before it 
<p style="text-align: right; clear: right;">6</p>

# Summary to this point

* Where:
     - *m* is the length of the text to be searched
     - *n* is the length of the pattern (maximum length if more than 1)
     - *p* is the number of patterns


|     Method    | Storage    | Single Search | Multi Search |
|:-------------:|:----------:|:-------------:|:------------:|
|  Brute Force  | O(m)       | O(nm)         | O(p n m)     |
| Keyword Tries | O(pn)      | O(nm)         | O(p m)       |
| Suffix Trees  | O(m)&ast;  | O(n)          | O(p n)       |
| Suffix Arrays | O(m)| O(n log(m))   | O(p n log(m))|


&ast; with large constants, however 
<p style="text-align: right; clear: right;">7</p>

# A rather unknown compression approach

In 1994, two researchers from DEC research labs in Palo Alto, Michael Burrows and David Wheeler, devised a transformation for text that made it more compressible. Essentially, they devised a *invertible permutation* of any text that compresses well if it exhibits redundancy.

**Example:**
<pre>
                   text = "amanaplanacanalpanama$"
              BWT(text) = "amnnn$lcpmnapaaaaaaala"
</pre>

* Notice how the transformed text has long *runs* of repeated characters
* A simple form of compression, called run-length encoding, replaces repeated symbols by a (count, symbol) tuple
* If the count is 1, then just the symbol appears

Thus, the BWT(text) can be represented as:
<pre>
              Compress(BWT(text)) = am3n$lcpmnap7ala  (16 chars instead of 22)
</pre>

* The savings are even more impressive for longer strings
* Notice, they introduced a special *"end-of-text"* symbol (&dollar; in our case), which is lexigraphically before any other symbol

<p style="text-align: right; clear: right;">8</p>

# Key Idea behind the BWT

* Sorting Cyclical Suffixes (say that 3-times fast)
<pre>
         "Cyclical Suffixes"         "Sorted Cyclical Suffixes" 
              tarheel$                    $tarhee<span style="color:blue;">l</span>
              arheel$t                    arheel$<span style="color:blue;">t</span>
              rheel$ta                    eel$tar<span style="color:blue;">h</span>
              heel$tar                    el$tarh<span style="color:blue;">e</span>
              eel$tarh                    heel$ta<span style="color:blue;">r</span>
              el$tarhe                    l$tarhe<span style="color:blue;">e</span>
              l$tarhee                    rheel$t<span style="color:blue;">a</span>
              $tarheel                    tarheel<span style="color:blue;">&dollar;</span>
</pre>

* The BWT of "tarheels" is the last column of the sorted cyclical suffixes "ltherea&dollar;"
* Notice that the sorted cyclical suffixes have a lot in common with a suffix array.
* The BWT is just the "predecessor symbol of these suffixes", where "&dollar;" precedes the first symbol 
<p style="text-align: right; clear: right;">9</p>

In [22]:
t="beatmiami$"
print [t[i:]+t[:i] for i in xrange(len(t))]

['beatmiami$', 'eatmiami$b', 'atmiami$be', 'tmiami$bea', 'miami$beat', 'iami$beatm', 'ami$beatmi', 'mi$beatmia', 'i$beatmiam', '$beatmiami']


# BWT in Python

* Straightforward implementation based on the definition (there are faster construction methods) 

In [23]:
def BWT(t):
    # create a sorted list of all cyclic suffixes of t
    rotation = sorted([t[i:]+t[:i] for i in xrange(len(t))])
    # concatenate the last symbols from each suffix
    return ''.join(r[-1] for r in rotation)

print BWT("banana$")
print BWT("amanaplanacanalpanama$")
print BWT("abananaban$")

annb$aa
amnnn$lcpmnapaaaaaaala
nn$bnbaaaaa


<p style="text-align: right; clear: right;">10</p>

# BWT from a Suffix Array

* It is even simpler to compute the BWT from a Suffix Array
* Finds each suffix's "predecessor" symbol 

In [24]:
def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)
    
newt =  t+'$'
sa = suffixArray(newt)
print newt
print sa
print BWTfromSuffixArray(newt, sa)

beatmiami$$
[10, 9, 6, 2, 0, 1, 8, 5, 7, 4, 3]
$iie$bmmata


<p style="text-align: right; clear: right;">11</p>

# Inverting a BWT

* A property of a transform is that there is no information loss-- they are invertible.

Algorithm: <u>inverseBWT(bwt)</u>
  1. Create a table of len(bwt) empty strings
  2. repeat length(*bwt*) times:
  3. &nbsp;&nbsp;&nbsp;&nbsp;prepend *bwt* as the first column of the table
  4. &nbsp;&nbsp;&nbsp;&nbsp;sort rows of the table alphabetically
  5. return (row of table with bwt's 'EOF' character)

<pre>
   <u>0</u>    <u>1</u>     <u>2</u>      <u>3</u>       <u>4</u>        <u>5</u>         <u>6</u>          <u>7</u>           <u>8</u>
   l    l$    l$t    l$ta    l$tar    l$tarh    l$tarhe    l$tarhee    $tarheel
   t    ta    tar    tarh    tarhe    tarhee    tarheel    tarheel$    arheel$t
   h    he    hee    heel    heel$    heel$t    heel$ta    heel$tar    eel$tarh
   e    ee    eel    eel$    eel$t    eel$ta    eel$tar    eel$tarh    el$tarhe
   r    rh    rhe    rhee    rheel    rheel$    rheel$t    rheel$ta    heel$tar
   e    el    el$    el$t    el$ta    el$tar    el$tarh    el$tarhe    l$tarhee
   a    ar    arh    arhe    arhee    arheel    arheel$    arheel$t    rheel$ta
   $    $t    $ta    $tar    $tarh    $tarhe    $tarhee    $tarheel    <span style="color:blue;">tarheel&dollar;</span>
</pre>

* What else do you notice about the final table?

<p style="text-align: right; clear: right;">12</p>

# Inverse BWT in Python

In [25]:
def inverseBWT(bwt):
    # initialize the table from t
    table = ['' for c in bwt]
    for j in xrange(len(bwt)):
        #insert the BWT as the first column
        table = sorted([c+table[i] for i, c in enumerate(bwt)])
    #return the row that ends with ‘$’
    return table[bwt.index('$')]

print inverseBWT("ltherea$")
print inverseBWT("amnnn$lcpmnapaaaaaaala")
print inverseBWT("annb$aa")
print inverseBWT("nn$bnbaaaaa")

tarheel$
amanaplanacanalpanama$
banana$
abananaban$


<p style="text-align: right; clear: right;">13</p>

# BWT Compression

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22BWTCompression.png" style="float: right;">
* Uncompressed the BWT(text) is same length as original text
* But, it has a tendancy to form long runs of repeated symbols
* Why does it form runs?
* All suffixes of repeated substrings sort together and share predecessors


* Somewhere further down the BWT there is a series of suffixes starting with *u's* that have *o's* as predecessors
* Redundancy leads to compression

<p style="text-align: right; clear: right;">14</p>

# What do BWTs have to do with searching strings? 

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22HiddenMessage.gif" style="float: right; margin: 10px 50px 10px 50px;">
* There is close relationship between BWTs and Suffix Arrays
* We can construct a suffix array from a BWT as we saw with InverseBWT(bwt)
* Is there a way to access this ***hidden suffix array*** for pattern searching?


* In 2005 two researchers, Ferragina & Manzini, figured out how
* First, an important property they uncovered

<p style="text-align: right; clear: right;">15</p>

# Last-First (LF) mapping property

* The BWT transforms "$banana\$$" to "$annb\$aa$"
* The predecessor symbols of a suffix array preserve the relative suffix order 
* The $j^{th}$ occurance of a symbol in the BWT corresponds to its $j^{th}$ occurance in the suffix array

<pre style="font-weight: bolder; font-size: 120%;">
          <span style="color:#00c0c0;">&dollar;</span>banan<span style="color:#c00000;">a</span>
          <span style="color:#c00000;">a</span>&dollar;bana<span style="color:#c08000;">n</span>
          <span style="color:#0000c0;">a</span>na&dollar;ba<span style="color:#c0c000;">n</span>
          <span style="color:#c000c0;">a</span>nana&dollar;<span style="color:#00c000;">b</span>
          <span style="color:#00c000;">b</span>anana<span style="color:#00c0c0;">&dollar;</span>
          <span style="color:#c08000;">n</span>a&dollar;ban<span style="color:#0000c0;">a</span>
          <span style="color:#c0c000;">n</span>ana&dollar;b<span style="color:#c000c0;">a</span>
</pre>

* This property allows one two traverse the suffix array indirectly
    - ex: The 1st "a" of the bwt is also the first "a" of the suffix array, and its predecessor is the 1st "n", whose predecessor is the 2nd "a", whose predecessor is the 2nd "n", and so on
* Meanwhile, the number of character occurences in the BWT matches the suffix array (recall it is a permutation)
<p style="text-align: right; clear: right;">16</p>

# The FM-index
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22FMIndex.png" style="float: right; margin: 10px 50px;">
* The FM-index is another *helper* data structure like the *LCP array* mentioned previously
* It is a 2D array whose size is $[|text|+1, |\Sigma|]$, where $|\Sigma|$ is the alphabet size
* It keeps track of how many of each symbol have been seen in the BWT prior to its $i^{th}$ symbol
* The last *m* row is the totals for each symbol. By accumulating these totals you can determine the BWT index corresponding to the first of each symbol in the suffix array (Offset).
* Can be generated by a single scan through the BWT
* Memory overhead $O(m|\Sigma|)$

<p style="text-align: right; clear: right;">17</p>

# Python for constructing FM-index

In [26]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].iteritems()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

bwt = "annb$aa"
FM, Offset = FMIndex(bwt)
print "BWT %2s,%2s,%2s,%2s" % tuple([symbol for symbol in sorted(Offset.keys())])
for i, row in enumerate(FM):
    print "%3s %2d,%2d,%2d,%2d" % tuple([bwt[i]+':' if i < len(bwt) else '']+[row[symbol] for symbol in sorted(row.keys())])
print [(sym, Offset[sym]) for sym in sorted(Offset.iterkeys())]

BWT  $, a, b, n
 a:  0, 0, 0, 0
 n:  0, 1, 0, 0
 n:  0, 1, 0, 1
 b:  0, 1, 0, 2
 $:  0, 1, 1, 2
 a:  1, 1, 1, 2
 a:  1, 2, 1, 2
     1, 3, 1, 2
[('$', 0), ('a', 1), ('b', 4), ('n', 5)]


<p style="text-align: right; clear: right;">18</p>

# Find a Suffix's Predecessor

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22FMIndex.png" style="float: right;">
* Given an index *i* in the BWT, find the index in the BWT of the suffix preceding the suffix represented by *i*
* Suffix 5 is preceded by suffix 2
* Suffix 2 is preceded by suffix 6
* Suffix 6 is preceded by suffix 3
* The predecessor suffix of index i:
``` python
        c = BWT[i]
        predec = Offset[c] + FMIndex[i][c]
```

* Predecessor of index 1
``` python
        c = BWT[1]                           # 'n'
        predec = O['n'] + FMIndex[1]['n']    # 5+0 = 5
```
* Predecessor of index 5
``` python
        c = BWT[5]                           # 'a'
        predec = O['a'] + FMindex[5]['a']    # 1+1 = 2
```
* Time to find predecessor: O(1)

<p style="text-align: right; clear: right;">19</p>

# Suffix Recovery

* What is the suffix array entry corresponding to BWT index *i*?
   - Start at *i* and repeatedly find predecessors until *i* is reached again

* To find the *original* string, just start with *i = 0*, the '$' index

In [27]:
def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

# recall that the FM-index that we built was "annb$aa", the BWT of "banana$"
for i in xrange(len(bwt)):
    print i, recoverSuffix(i, bwt, FM, Offset)

0 $banana
1 a$banan
2 ana$ban
3 anana$b
4 banana$
5 na$bana
6 nana$ba


<p style="text-align: right; clear: right;">20</p>

# Finding Substrings

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22FMIndex.png" style="float: right;">
* Searches are performed in reverse order
* Searches return an interval of the suffix array that starts with the desired substring
    - Finds all occurences of target
    - If there are no occurences it finds an empty interval
* Starts with full BWT range (0, N)
* Narrows the range one symbol at a time
* To find substring "nana"
``` python
    # Initialize to full range of suffix array
    lo, hi = 0, len(BWT)                      # len(BWT) = 7
    # Find occurrences of "a"
    lo = Offset['a'] + FMIndex[lo]['a']       # lo = 1 + 0 = 1
    hi = Offset['a'] + FMIndex[hi]['a']       # hi = 1 + 3 = 4
    # Find occurrences of "na"
    lo = Offset['n'] + FMIndex[lo]['n']       # lo = 5 + 0 = 5
    hi = Offset['n'] + FMIndex[hi]['n']       # hi = 5 + 2 = 7
    # Find occurrences of "ana"
    lo = Offset['a'] + FMIndex[lo]['a']       # lo = 1 + 1 = 2
    hi = Offset['a'] + FMIndex[hi]['a']       # hi = 1 + 3 = 4
    # Find occurrences of "nana"
    lo = Offset['n'] + FMIndex[lo]['n']       # lo = 5 + 1 = 6
    hi = Offset['n'] + FMIndex[hi]['n']       # hi = 5 + 2 = 7
```
<p style="text-align: right; clear: right;">21</p>

# In Python

* One of the simplest methods we've seen for searching

In [28]:
def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi

print findBWT("ana", FM, Offset)
print findBWT("ban", FM, Offset)
print findBWT("ann", FM, Offset)

(2, 4)
(4, 5)
(4, 4)


<p style="text-align: right; clear: right;">22</p>

# BWT score card

|     Method    | Storage    | Single Search | Multi Search |
|:-------------:|:----------:|:-------------:|:------------:|
|  Brute Force  | O(m)       | O(nm)         | O(p n m)     |
| Keyword Tries | O(pn)      | O(nm)         | O(p m)       |
| Suffix Trees  | O(m)&ast;  | O(n)          | O(p n)       |
| Suffix Arrays | O(m log(m))| O(n log(m))   | O(p n log(m))|
|     BWT       | O(m)&dagger; | O(n)          | O(p n)       |

Where:
* *m* is the length of the text to be searched
* *n* is the length of the pattern (maximum length if more than 1)
* *p* is the number of patterns

&ast; With large constants, however<br>&dagger; Usually significantly smaller than *m*
<p style="text-align: right; clear: right;">23</p>

# BWT Gotchas

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22SampledFMIndex.png" style="float: right;">
* While the BWT itself is small, its FM-index can be large
* A full FM-index requires *O(|&Sigma;| m)* space
* But it can be sampled with minimal performance impact
    - rather than store the FM-index for all indices store only 1 in F
    - when accessing find the closest smaller instantiated index<br> and use the BWT to fill in the requested missing values
* Example with F = 3
    - when FMIndex[5]['b'] is accessed
    - retrive FMIndex[3]['b'] = 0
    - scan BWT from [3:5] counting 'b's (1) and adding them to the count at FMIndex[3]
    - return the count = 1
* In practice F values as large as 1000 have little performance impact
    - Why? BWT is small and tends to stay in cache
    - BWT is compressed so scanning through 1000 characters involves fewer reads 

* To have all the capabilities of a Suffix Tree, a BWT needs an LCP array
<p style="text-align: right; clear: right;">24</p>

# Real-World uses of BWTs

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22BWTAligner.png" style="float: right;">
BWTs are the dominant representation and method used for ***Sequence Alignment***

* <u>Sequence-Alignment Problem:</u> Given a collection of short nucleotide fragments (either DNA or RNA) find the best approximate alignment for each read in a reference genome
* Bowtie2 (2012) and BWA (2009) are the dominant aligners
* As a preprocess a BWT of the reference genome is built (&approx; 1-3 GB)
* Alignment:
    * given a *read* from a sequenced fragment (72-150 base pairs typically)
    * cut the read into smaller seeds (25-31 base pairs typically)
    * Search for an exact match to each using the BWT
    * Use local alignment (dynamic program) to match the remaining bases
<p style="text-align: right; clear: right;">25</p>

# Next Time

### We go deeper down the BWT rabbit hole
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/L22RabbitHole.jpg" width="640px">
<p style="text-align: right; clear: right;">26</p>

In [35]:
text = "imissmissmississippiskiss$"
bwt = BWT(text)
print text
print bwt
print inverseBWT(bwt)
print
FM, Offset = FMIndex(bwt)

for i in xrange(len(bwt)):
    print "%2d %s" % (i, recoverSuffix(i, bwt, FM, Offset))

print
print findBWT("iss", FM, Offset)
print findBWT("ssi", FM, Offset)

imissmissmississippiskiss$
s$spksmmmsssipisssissiiiii
imissmissmississippiskiss$

 0 $imissmissmississippiskiss
 1 imissmissmississippiskiss$
 2 ippiskiss$imissmissmississ
 3 iskiss$imissmissmississipp
 4 iss$imissmissmississippisk
 5 issippiskiss$imissmissmiss
 6 ississippiskiss$imissmissm
 7 issmississippiskiss$imissm
 8 issmissmississippiskiss$im
 9 kiss$imissmissmississippis
10 mississippiskiss$imissmiss
11 missmississippiskiss$imiss
12 missmissmississippiskiss$i
13 piskiss$imissmissmississip
14 ppiskiss$imissmissmississi
15 s$imissmissmississippiskis
16 sippiskiss$imissmissmissis
17 sissippiskiss$imissmissmis
18 skiss$imissmissmississippi
19 smississippiskiss$imissmis
20 smissmississippiskiss$imis
21 ss$imissmissmississippiski
22 ssippiskiss$imissmissmissi
23 ssissippiskiss$imissmissmi
24 ssmississippiskiss$imissmi
25 ssmissmississippiskiss$imi

(4, 9)
(22, 24)
